# Тестовое задание – Ганнова Анфиса

In [1]:
import re
import pandas as pd

# Задания 1

In [4]:
df1_1 = pd.read_excel('Тестовое_задание_аналитик.xlsx', skiprows=3)

In [5]:
display(df1_1)

,Data,id1,id2,id3,id4,id5
0,[FAN TOP CPL AU V][##1699]_[rAU][c8][eEUR],FAN TOP CPL AU V,1699,AU,8.0,EUR
1,"[GRTM2 CPC DE A][##961]_[rDE][c3,5][eUSD]",GRTM2 CPC DE A,961,DE,3.5,USD
2,[EURCAR CPA PL I V][##67]_[rPL][c155][eEUR],EURCAR CPA PL I V,67,PL,155.0,EUR
3,[Win Play RU U][##5]_[rRU][c1250][eRUB],Win Play RU U,5,RU,1250.0,RUB
4,[VIPMUS4 DECH A G][##11021]_[rDECH][c80][eUSD],VIPMUS4 DECH A G,11021,DECH,80.0,USD


In [9]:
# дублируем таблицу - первый столбец
df1_2 = df1_1.copy().iloc[:,[0]]

In [13]:
def parse_data_string_corrected(data_string):
    """
    Разбиваем строку
    """
    if pd.isna(data_string) or not isinstance(data_string, str):
        return ['', '', '', '', '']
    
    # Убираем кавычки если есть
    data_string = data_string.strip().strip('"')
    
    # Паттерн для разбора: [компонент1][##число]_[rстрана][cкатегория][eвалюта]
    pattern = r'\[([^\]]+)\]\[##(\d+)\]_\[r([^\]]+)\]\[c([^\]]+)\]\[e([^\]]+)\]'
    
    match = re.match(pattern, data_string.strip())
    
    if match:
        component1 = match.group(1).strip()  # Компонент 1
        number = match.group(2)              # Номер
        country = match.group(3)             # Код страны
        category_raw = match.group(4)        # Категория (может содержать запятую)
        currency = match.group(5)            # Валюта
        
        # Обрабатываем категорию
        if ',' in category_raw:
            # Если есть запятая, заменяем на точку
            category_formatted = category_raw.replace(',', '.')
        else:
            # Если целое число, добавляем .0
            try:
                int_val = int(category_raw)
                category_formatted = f"{int_val}.0"
            except ValueError:
                category_formatted = category_raw
        
        result = [component1, number, country, category_formatted, currency]
        return result
    
    else:
        print(f"  Паттерн не совпал для: {data_string}")
        return ['', '', '', '', '']


def process_dataframe(df, data_column='Data'):
    '''
     Создание столбцов
    '''
    # Применяем парсинг к каждой строке
    parsed_data = df[data_column].apply(parse_data_string_corrected)
    
    # Добавляем новые столбцы
    df['id1'] = [item[0] for item in parsed_data]  # Компонент 1
    df['id2'] = [item[1] for item in parsed_data]  # Номер
    df['id3'] = [item[2] for item in parsed_data]  # Код страны
    df['id4'] = [item[3] for item in parsed_data]  # Категория
    df['id5'] = [item[4] for item in parsed_data]  # Валюта
    
    return df


df1_2 = process_dataframe(df1_2)    

## Результат

In [14]:
display(df1_2)

,Data,id1,id2,id3,id4,id5
0,[FAN TOP CPL AU V][##1699]_[rAU][c8][eEUR],FAN TOP CPL AU V,1699,AU,8.0,EUR
1,"[GRTM2 CPC DE A][##961]_[rDE][c3,5][eUSD]",GRTM2 CPC DE A,961,DE,3.5,USD
2,[EURCAR CPA PL I V][##67]_[rPL][c155][eEUR],EURCAR CPA PL I V,67,PL,155.0,EUR
3,[Win Play RU U][##5]_[rRU][c1250][eRUB],Win Play RU U,5,RU,1250.0,RUB
4,[VIPMUS4 DECH A G][##11021]_[rDECH][c80][eUSD],VIPMUS4 DECH A G,11021,DECH,80.0,USD


<div style="background-color: white; height: 9rem;"></div>


# Задание 2

In [200]:
#  импорт данных
df2 = pd.read_csv('тест (данные).csv')

In [201]:
display(df2.sample(3))

,timestamp,target id,target store id,target name,creative pack id,creative pack name,campaign id,campaign name,starts,views,clicks,installs,spend,cvr,ctr,ecpm,cpi
29,2018-10-08T00:00:00.000Z,500011320,1424128078,BetterMen: Workout Trainer,5bb393cc9964fe0045f767ac,betterme_02.10_q,5bb386c78d569c0044e24f09,BetterMen_ios_US_1,1,0,0,2,2.40,2.000000,0.0000,2400.00,1.20
16,2018-10-01T00:00:00.000Z,500011320,1424128078,BetterMen: Workout Trainer,5bb394188d569c0044e24f4b,betterme_02.10_w,5bb386c78d569c0044e24f09,BetterMen_ios_US_1,6182,5006,279,21,26.20,0.003397,0.0451,4.24,1.25
20,2018-10-08T00:00:00.000Z,500009477,1405925081,Tascee App,5b83bfb05732d6004ee26668,g3,5b7ff179ae82bd004a3c4f09,37_441_appcent,194828,167164,5439,1875,938.05,0.009624,0.0279,4.81,0.50


In [202]:
# очищаем данные
def df_clean(df):
    # 1. Преобразование timestamp в читабельный формат
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['date'] = df['timestamp'].dt.strftime('%d.%m.%Y')
    
    # 2. Очистка числовых данных
    numeric_columns = ['views', 'clicks', 'installs', 'spend', 'cvr', 'ctr', 'ecpm', 'cpi']
    for col in numeric_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
    
    # 3. Расчет SPI (Spend Per Install)
    df['spi'] = np.where(df['installs'] > 0, df['spend'] / df['installs'], 0)
    
    # 4. Обработка бесконечных значений
    df = df.replace([np.inf, -np.inf], 0)
    
    df = df.rename(columns={
        'target id': 'Target ID',
        'target store id': 'Target Store ID',
        'target name': 'Target Name',
        'creative pack id': 'Creative Pack ID',
        'creative pack name': 'Creative Pack Name',
        'campaign id': 'Campaign ID',
        'campaign name': 'Campaign Name',
        'starts': 'Starts',
        'views': 'Views',
        'clicks': 'Clicks',
        'installs': 'Installs',
        'spend': 'Spend',
        'cvr': 'CVR',
        'ctr': 'CTR',
        'ecpm': 'ECPM',
        'cpi': 'CPI',
        'date': 'Date',
        'spi': 'SPI'
    })
    return df

df2 = df_clean(df2)

In [203]:
# сводная таблица
pivot_table = pd.pivot_table(df2,
               index=["Date", "Target Name"],
               values=['Views', 'Clicks', 'Installs', 'Spend', 'SPI'],
               aggfunc={'Views': 'sum', 'Clicks': 'sum', 'Installs': 'sum', 'Spend': 'sum', 'SPI': 'mean'})

In [204]:
display(pivot_table)

Clicks  Installs       SPI  \
Date       Target Name                                                  
01.10.2018 BetterMe: Weight Loss Workouts     432        13  0.752381   
           BetterMen: Workout Trainer         720        72  1.180317   
           Gemini Photos: Gallery Cleaner     252        70  0.500000   
           PROFI.RU                           364         9  1.250000   
           Tascee App                       10081      3365  0.514819   
           Words of Clans - word game         399        26  0.500000   
08.10.2018 BetterMe: Weight Loss Workouts       8         3  0.583333   
           BetterMen: Workout Trainer           0         4  1.100000   
           Gemini Photos: Gallery Cleaner       0         0  0.000000   
           PROFI.RU                            52         2  1.250000   
           Tascee App                        5497      1886  0.502419   
           Words of Clans - word game         164         7  0.500000   

                                             Spend   Views  
Date       Target Name                                      
01.10.2018 BetterMe: Weight Loss Workouts    15.35   10866  
           BetterMen: Workout Trainer        84.40   15090  
           Gemini Photos: Gallery Cleaner    35.00    7174  
           PROFI.RU                          11.25   10260  
           Tascee App                      1628.75  297682  
           Words of Clans - word game        13.00    5575  
08.10.2018 BetterMe: Weight Loss Workouts     3.50     219  
           BetterMen: Workout Trainer         4.40       0  
           Gemini Photos: Gallery Cleaner     0.00       1  
           PROFI.RU                           2.50     754  
           Tascee App                       943.60  169383  
           Words of Clans - word game         3.50    2142

In [205]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side

# Функция для форматирования дат в краткий вид
def format_dates_short(df):
    """Преобразует столбцы с датами в краткий строковый формат"""
    df_copy = df.copy()
    
    for col in df_copy.columns:
        # Проверяем на datetime с timezone
        if pd.api.types.is_datetime64tz_dtype(df_copy[col]):
            df_copy[col] = df_copy[col].dt.tz_localize(None)
        
        # Проверяем на обычный datetime
        if pd.api.types.is_datetime64_any_dtype(df_copy[col]):
            # Форматируем дату в краткий вид (например, 2024-01-15)
            df_copy[col] = df_copy[col].dt.strftime('%Y-%m-%d')
    
    return df_copy


def format_excel_sheet(writer, sheet_name, df, is_pivot=False):
    """Применяет красивое форматирование к листу Excel,
       учитывая многоуровневые заголовки сводной таблицы."""
    workbook = writer.book
    worksheet = writer.sheets[sheet_name]

    header_font = Font(bold=True, color="FFFFFF")
    header_fill = PatternFill(start_color="366092", end_color="366092", fill_type="solid")
    header_alignment = Alignment(horizontal="center", vertical="center")
    thin_border = Border(
        left=Side(style='thin'),
        right=Side(style='thin'),
        top=Side(style='thin'),
        bottom=Side(style='thin')
    )

    # Определяем, сколько строк занимают заголовки
    # Для сводной таблицы с мультииндексами это может быть больше 1
    num_header_rows = 1
    if is_pivot and isinstance(df.columns, pd.MultiIndex):
        num_header_rows = len(df.columns.levels)

    # Применяем стили к заголовкам
    for row_num in range(1, num_header_rows + 1):
        for col_num in range(1, len(df.columns) + len(df.index.names) + 1):
            cell = worksheet.cell(row=row_num, column=col_num)
            cell.font = header_font
            cell.fill = header_fill
            cell.alignment = header_alignment
            cell.border = thin_border
    
    # ... (остальная часть вашей функции без изменений)
    # Автоподбор ширины столбцов
    for col_num in range(1, len(df.columns) + len(df.index.names) + 1):
        column_letter = worksheet.cell(row=1, column=col_num).column_letter
        max_length = len(str(worksheet.cell(row=1, column=col_num).value))
        
        # Здесь логика автоподбора должна быть пересмотрена
        # из-за multi-index заголовков.
        # Например, можно просто поставить стандартную ширину, если расчет сложный.
        worksheet.column_dimensions[column_letter].width = 15
        
    # Применяем границы ко всем ячейкам с данными
    # Учитываем, что данные начинаются ниже заголовков
    for row in range(1, len(df) + num_header_rows + 1):
        for col in range(1, len(df.columns) + len(df.index.names) + 1):
            worksheet.cell(row=row, column=col).border = thin_border
# Функция для красивого форматирования листа Excel
# def format_excel_sheet(writer, sheet_name, df):
#     """Применяет красивое форматирование к листу Excel"""
#     workbook = writer.book
#     worksheet = writer.sheets[sheet_name]
    
#     # Стили для заголовков
#     header_font = Font(bold=True, color="FFFFFF")
#     header_fill = PatternFill(start_color="366092", end_color="366092", fill_type="solid")
#     header_alignment = Alignment(horizontal="center", vertical="center")
    
#     # Стили для границ
#     thin_border = Border(
#         left=Side(style='thin'),
#         right=Side(style='thin'),
#         top=Side(style='thin'),
#         bottom=Side(style='thin')
#     )
    
#     # Применяем стили к заголовкам
#     for col_num in range(1, len(df.columns) + 1):
#         cell = worksheet.cell(row=1, column=col_num)
#         cell.font = header_font
#         cell.fill = header_fill
#         cell.alignment = header_alignment
#         cell.border = thin_border
        
#         # Автоподбор ширины столбцов
#         column_letter = cell.column_letter
#         max_length = max(
#             len(str(df.columns[col_num-1])),  # длина заголовка
#             df.iloc[:, col_num-1].astype(str).str.len().max() if not df.empty else 0  # максимальная длина данных
#         )
#         adjusted_width = min(max_length + 2, 50)  # +2 для отступов, максимум 50
#         worksheet.column_dimensions[column_letter].width = adjusted_width
    
#     # Применяем границы ко всем ячейкам с данными
#     for row in range(1, len(df) + 2):
#         for col in range(1, len(df.columns) + 1):
#             worksheet.cell(row=row, column=col).border = thin_border
#             # Выравнивание для ячеек с данными
#             if row > 1:
#                 worksheet.cell(row=row, column=col).alignment = Alignment(horizontal="left", vertical="center")

df2 = df2.drop('timestamp', axis=1)
# 2. Перемещаем столбец 'Date' в начало
# Получаем список всех столбцов
cols = df2.columns.tolist()
# Удаляем 'Date' из текущего места
cols.remove('Date')
# Вставляем 'Date' в начало
cols.insert(0, 'Date')

# Применяем новый порядок к DataFrame
df2 = df2[cols]

# Подготавливаем данные с красиво отформатированными датами
df2 = format_dates_short(df2)
pivot_table_formatted = format_dates_short(pivot_table)

with pd.ExcelWriter('Тестовое_задание_аналитик.xlsx', engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    
    # Задание 2.1 — обычная таблица
    df2.to_excel(writer, sheet_name='Задание_2.1', index=False, header=True, startrow=0, startcol=0)
    format_excel_sheet(writer, 'Задание_2.1', df2)
    
    # Задание 2.2 — сводная таблица с мультииндексами
    pivot_table_formatted.to_excel(
        writer,
        sheet_name='Задание_2.2',
        index=True,               # сохраняем индексы
        header=True,            
        startrow=0,
        startcol=0,
        merge_cells=True          
    )
    format_excel_sheet(writer, 'Задание_2.2', pivot_table_formatted)


<div style="background-color: white; height: 9rem;"></div>


# Задания 3

1. Из листов в Гугл таблице собрать все данные на одном листе (выполнить в своей Гугл таблице, с помощью формулы, не вручную!)
2. На отдельном листе сделать маленькую таблицу, собирающую в себе итоговые данные по деятельности команд в реальном времени (Команда/Потраченная сумма/итог общий)
Расчет потраченной суммы производить с помощью формулы, не использовать сводную таблицу

In [221]:
df3_1 = pd.read_excel('Test22.xlsx', sheet_name='TOP')
df3_2 = pd.read_excel('Test22.xlsx', sheet_name='Kotiki')
df3_3 = pd.read_excel('Test22.xlsx', sheet_name='Командочка')

In [222]:
display(df3_1.head(2))

,name,gender,origin,Team,Cost USD
0,Aaleyah,Female,Arabic,TOP,172
1,Aaliyah,Female,Arabic,TOP,22000


In [223]:
display(df3_2.head(2))

,name,gender,origin,Team,Cost USD
0,Aahan,Male,Hindi,Kotiki,236
1,Aaleahya,Female,Indian,Kotiki,17


In [224]:
display(df3_3.head(2))

,name,gender,origin,Team,Cost USD
0,Aala,Female,American,Командочка,2000
1,Aalijah,Female,Hebrew,Командочка,64


In [225]:
df3 = pd.concat([df3_1, df3_2, df3_3], ignore_index=True)

In [226]:
summary = df3.groupby('Team')['Cost USD'].agg(['sum', 'count']).reset_index()
summary.columns = ['Команда', 'Потраченная сумма', 'Количество участников']

# Добавляем общий итог
total_sum = summary['Потраченная сумма'].sum()
total_count = summary['Количество участников'].sum()

# Добавляем строку с итогом
summary_with_total = summary.copy()
summary_with_total.loc[len(summary_with_total)] = ['ИТОГО', total_sum, total_count]
 

In [227]:
summary_with_total

,Команда,Потраченная сумма,Количество участников
0,Kotiki,875865792,12291
1,TOP,170906078,4903
2,Командочка,28661500,610
3,ИТОГО,1075433370,17804


In [228]:
with pd.ExcelWriter('Тестовое_задание_аналитик.xlsx', engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    
    # Записываем df1 на новый лист 'Таблица_1'
    df3.to_excel(writer, sheet_name='Задание_3.1', index=False)
    
    # Записываем df2 на новый лист 'Таблица_2'
    summary_with_total.to_excel(writer, sheet_name='Задание_3.2', index=False)
    